## 문제1
- 데이터셋(basic1.csv)의 'f5' 컬럼을 기준으로 상위 10개의 데이터를 구하고,
- 'f5'컬럼 10개 중 최소값으로 데이터를 대체한 후, 
- 'age'컬럼에서 80 이상인 데이터의'f5 컬럼 평균값 구하기

In [ ]:
# 라이브러리 및 데이터 불러오기
import pandas as pd

df = pd.read_csv('../input/bigdatacertificationkr/basic1.csv')
df.head()

In [ ]:
df = df.sort_values('f5', ascending=False)
df10 = df['f5'][:10].min()
df.iloc[:10, -1] = df10
age80 = df[df['age']>=80]['f5'].mean()
age80

## 문제2
- 데이터셋(basic1.csv)의 앞에서 순서대로 70% 데이터만 활용해서,
- 'f1'컬럼 결측치를 중앙값으로 채우기 전후의 표준편차를 구하고
- 두 표준편차 차이 계산하기

In [ ]:
# 라이브러리 및 데이터 불러오기
import pandas as pd
import numpy as np

df = pd.read_csv('../input/bigdatacertificationkr/basic1.csv')

In [ ]:
df70 = df.iloc[:70]
df.isnull().sum()
std1 = df['f1'].std()
med = df70['f1'].median()
df70['f1'] = df70['f1'].fillna(med)
std2 = df70['f1'].std()
print(std1 - std2)

## 문제3
- 데이터셋(basic1.csv)의 'age'컬럼의 이상치를 더하시오!
- 단, 평균으로부터 '표준편차*1.5'를 벗어나는 영역을 이상치라고 판단함

In [ ]:
# 라이브러리 및 데이터 불러오기
import pandas as pd
import numpy as np

df = pd.read_csv('../input/bigdatacertificationkr/basic1.csv')

In [ ]:
avg = df['age'].mean()
std1 = df['age'].std()*1.5
min = avg - std1
max = avg + std1
out = df[(df['age']>max)|(df['age']<min)]['age'].sum()
print(out)

## 2유형 - 전자상거래 배송 데이터
### 제품 배송 시간에 맞춰 배송되었는지 예측모델 만들기
학습용 데이터 (X_train, y_train)을 이용하여 배송 예측 모형을 만든 후, 이를 평가용 데이터(X_test)에 적용하여 얻은 예측 확률값을 다음과 같은 형식의 CSV파일로 생성하시오(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점)
```
ID, Reached.on.Time_Y.N
4733,0.6
2040,0.8
5114,0.45
2361,0.23
5996,0.43
```

[시험용 데이터셋 만들기] 코드는 예시문제와 동일한 형태의 X_train, y_train, X_test 데이터를 만들기 위함임

(유의사항)
- 성능이 우수한 예측모형을 구축하기 위해서는 적절한 데이터 전처리, 피처엔지니어링, 분류알고리즘, 하이퍼파라미터 튜닝, 모형 앙상블 등이 수반되어야 한다.
- 수험번호.csv파일이 만들어지도록 코드를 제출한다.
- 제출한 모델의 성능은 ROC-AUC형태로 읽어드린다.

In [ ]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("../input/customer-analytics/Train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Reached.on.Time_Y.N', id_name='ID')

X_train.shape, X_test.shape, y_train.shape, y_test.shape


2. EDA

In [ ]:
# 제공된 3개 데이터 (y_test데이터 활용X)
X_train.shape, X_test.shape, y_train.shape

In [ ]:
x_train = X_train
x_test = X_test

In [ ]:
x_train.info(), x_test.info(), y_train.info()

In [ ]:
# 원핫인코딩 대상 파악
print(x_train.describe(include='object'))
print(x_test.describe(include='object'))

3. 전처리 및 분리

In [ ]:
# 원핫인코딩
x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)

print(x_train.info())

In [ ]:
# 데이터 분리
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train,
                                                 y_train['Reached.on.Time_Y.N'], 
                                                 test_size=0.2,
                                                 stratify=y_train['Reached.on.Time_Y.N'],
                                                 random_state=2023)
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

4. 모델링 및 성능평가

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=2023)
model.fit(x_train, y_train)

In [ ]:
# 모델을 사용하여 테스트 데이터 예측
y_pred = model.predict(x_val)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
acc = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)
roc = roc_auc_score(y_val, y_pred)

print(acc)
print(f1)
print(roc)

# 0.6755681818181818
# 0.69676048858205
# 0.6877330650570087

In [ ]:
y_result = model.predict(x_test)
print(y_result)

# [0 1 1 ... 0 0 0]

In [ ]:
result = pd.DataFrame({
        'cust_id': x_test['ID'],
        'gender': y_result
    })
result.to_csv('003000.csv', index=False)